In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# !rm -rf /content/scala_label
!rm -rf /content/scala_edit_label

In [ ]:
import os
import shutil

source_dir = "/content/drive/MyDrive/Goruntu_Isleme/Celik/dokme_demir"
destination_dir = "/content/scala_label"
os.makedirs(destination_dir, exist_ok=True)

for filename in os.listdir(source_dir):
    if filename.endswith(".json"):
        source_path = os.path.join(source_dir, filename)
        destination_path = os.path.join(destination_dir, filename)
        shutil.copy2(source_path, destination_path)


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/Goruntu_Isleme/Celik/dokme_demir'

In [ ]:
import os
import json
import requests
from io import BytesIO
from PIL import Image
import matplotlib.pyplot as plt

# 📂 Giriş ve çıkış klasörleri
json_folder = '/content/scala_label'
save_folder = '/content/scala_originals'
os.makedirs(save_folder, exist_ok=True)

# 📋 Tüm .json dosyalarını al
json_files = [f for f in os.listdir(json_folder) if f.endswith('.json')]

# 🔁 Her dosya için görselleri göster ve orijinali kaydet
for json_file in json_files:
    json_path = os.path.join(json_folder, json_file)

    try:
        with open(json_path, 'r', encoding='utf-8') as f:
            task = json.load(f)

        # URL'leri al
        original_url = task['attachmentS3Downloads'][0]['s3URL']
        mask_url = task['response']['annotations']['combined']['image']
        filename = task['metadata']['filename']
        filename_base = os.path.splitext(filename)[0]

        # Görselleri indir
        original_img = Image.open(BytesIO(requests.get(original_url).content)).convert('RGB')
        mask_img = Image.open(BytesIO(requests.get(mask_url).content)).convert('RGB')

        # 📊 Görselleri yan yana göster
        fig, axs = plt.subplots(1, 2, figsize=(12, 6))
        axs[0].imshow(original_img)
        axs[0].set_title('Orijinal Görüntü')
        axs[0].axis('off')

        axs[1].imshow(mask_img)
        axs[1].set_title('Maske (combined.image)')
        axs[1].axis('off')

        plt.suptitle(filename, fontsize=14)
        plt.tight_layout()
        plt.show()

        # 💾 Sadece orijinal görüntüyü kaydet
        save_path = os.path.join(save_folder, f"{filename_base}.png")
        original_img.save(save_path)

        print(f"✅ Kaydedildi: {save_path}")

    except Exception as e:
        print(f"❌ Hata ({json_file}): {e}")


# Dosyalara kaydetme

In [ ]:
import os
import json
import requests
from io import BytesIO
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import cv2

# Ayarlar
EPSILON_FACTOR = 0.0001
MIN_POLYGON_AREA = 10

# Klasörler
input_dir = '/content/scala_label'
json_output_dir = '/content/scala_edit_label'
image_output_dir = '/content/scala_edit_image'
os.makedirs(json_output_dir, exist_ok=True)
os.makedirs(image_output_dir, exist_ok=True)

# Yardımcı fonksiyonlar
def hex_to_rgb(hex_color):
    hex_color = hex_color.lstrip('#')
    return tuple(int(hex_color[i:i+2], 16) for i in (0, 2, 4))

def find_closest_label(rgb_color, label_map, tolerance=10):
    rgb_arr = np.array(rgb_color)
    for ref_rgb, label in label_map.items():
        if np.linalg.norm(rgb_arr - np.array(ref_rgb)) <= tolerance:
            return label
    return None

# Tüm JSON dosyaları
json_files = [f for f in os.listdir(input_dir) if f.endswith('.json')]

for json_file in json_files:
    try:
        json_path = os.path.join(input_dir, json_file)
        with open(json_path, 'r', encoding='utf-8') as f:
            task = json.load(f)

        filename = task['metadata']['filename']
        width = task['attachmentS3Downloads'][0]['metadata']['width']
        height = task['attachmentS3Downloads'][0]['metadata']['height']
        label_mapping = task['response']['labelMapping']
        original_url = task['attachmentS3Downloads'][0]['s3URL']
        mask_url = task['response']['annotations']['combined']['image']

        original_img = Image.open(BytesIO(requests.get(original_url).content)).convert('RGB')
        mask_img = Image.open(BytesIO(requests.get(mask_url).content)).convert('RGB')

        original_array = np.array(original_img)
        mask_array = np.array(mask_img)

        rgb_to_label = {hex_to_rgb(info['color']): name for name, info in label_mapping.items()}
        overlay = np.zeros((height, width, 4), dtype=np.uint8)
        results = []

        unique_colors = np.unique(mask_array.reshape(-1, 3), axis=0)

        for color in unique_colors:
            rgb = tuple(color.tolist())
            phase_name = find_closest_label(rgb, rgb_to_label)

            if not phase_name:
                print(f"⚠️ Eşleşmeyen renk: {rgb} in {json_file}")
                continue

            binary_mask = np.all(mask_array == rgb, axis=-1).astype(np.uint8) * 255
            contours, _ = cv2.findContours(binary_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

            for cnt in contours:
                if cv2.contourArea(cnt) < MIN_POLYGON_AREA:
                    continue

                epsilon = EPSILON_FACTOR * cv2.arcLength(cnt, True)
                approx = cv2.approxPolyDP(cnt, epsilon, True)
                polygon = approx.squeeze().tolist()

                if isinstance(polygon[0], int):
                    polygon = [polygon]
                poly_pts = np.array(polygon, dtype=np.int32)

                polygon_mask = np.zeros_like(binary_mask)
                cv2.drawContours(polygon_mask, [poly_pts], -1, 255, thickness=cv2.FILLED)
                final_mask = cv2.bitwise_and(binary_mask, polygon_mask)
                coords = np.column_stack(np.where(final_mask == 255))
                for y, x in coords:
                    overlay[y, x] = (*rgb, 127)

                cv2.drawContours(overlay, [poly_pts], -1, (*rgb, 127), thickness=2)

                xs = [p[0] for p in polygon]
                ys = [p[1] for p in polygon]
                rect_mask = {
                    "xMin": min(xs),
                    "yMin": min(ys),
                    "width": max(xs) - min(xs),
                    "height": max(ys) - min(ys)
                }
                label_location = {"x": np.mean(xs), "y": np.mean(ys)}

                result = {
                    "content": [{"x": float(p[0]), "y": float(p[1])} for p in polygon],
                    "rectMask": rect_mask,
                    "labels": {
                        "labelName": phase_name.capitalize(),
                        "labelColor": '#%02x%02x%02x' % rgb,
                        "labelColorRGB": ",".join(str(c) for c in rgb),
                        "visibility": False
                    },
                    "labelLocation": label_location,
                    "contentType": "polygon"
                }

                results.append(result)

        combined_image = cv2.addWeighted(original_array, 1.0, overlay[:, :, :3], 0.5, 0)
        combined_image_rgb = cv2.cvtColor(combined_image, cv2.COLOR_BGR2RGB)

        # 🎨 Legend
        legend_patches = []
        for ref_rgb, label in rgb_to_label.items():
            normalized_rgb = tuple(np.array(ref_rgb) / 255)
            patch = mpatches.Patch(color=normalized_rgb, label=label.capitalize())
            legend_patches.append(patch)

        # 📊 Göster
        plt.figure(figsize=(10, 8))
        plt.imshow(combined_image)
        plt.title(f"Overlay: {filename}")
        plt.axis('off')
        plt.legend(handles=legend_patches, loc='lower right', fontsize=12, frameon=True)
        plt.tight_layout()
        plt.show()

        # 🔽 Kaydet
        filename_clean = filename.replace('.png', '').replace('.jpg', '')

        json_path_out = os.path.join(json_output_dir, f"{filename_clean}.json")
        image_path_out = os.path.join(image_output_dir, f"{filename_clean}_overlay.png")

        with open(json_path_out, 'w', encoding='utf-8') as f:
            json.dump(results, f, indent=4)
        Image.fromarray(combined_image).save(image_path_out)

        print(f"✅ Kaydedildi: {json_file} → JSON + Görsel")

    except Exception as e:
        print(f"❌ Hata ({json_file}): {e}")


# Test

In [ ]:
import os
import json
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np

# Klasör yolları
json_folder = '/content/scala_edit_label'
image_folder = '/content/scala_originals'

# .json dosyalarını bul
json_files = [f for f in os.listdir(json_folder) if f.endswith('.json')]

for json_file in json_files:
    try:
        json_path = os.path.join(json_folder, json_file)

        with open(json_path, 'r', encoding='utf-8') as f:
            polygon_data = json.load(f)

        # Dosya adından görüntü adını çıkar
        image_name = json_file.replace('.json', '')
        image_path = os.path.join(image_folder, f"{image_name}.png")

        if not os.path.exists(image_path):
            print(f"❌ Görsel bulunamadı: {image_path}")
            continue

        # Görseli yükle
        image = Image.open(image_path)
        image_array = np.array(image)

        # Görseli göster
        fig, ax = plt.subplots(figsize=(10, 8))
        ax.imshow(image_array)
        ax.set_title(f"Nokta Çizimi: {image_name}")
        ax.axis('off')

        # Poligonları çiz
        for item in polygon_data:
            coords = item.get('content', [])
            label_info = item.get('labels', {})
            hex_color = label_info.get('labelColor', '#ff0000')  # varsayılan kırmızı

            xs = [p['x'] for p in coords]
            ys = [p['y'] for p in coords]
            ax.plot(xs, ys, marker='o', markersize=2, linestyle='-', linewidth=1, color=hex_color)

        plt.tight_layout()
        plt.show()

    except Exception as e:
        print(f"❌ Hata ({json_file}): {e}")
